In [ ]:
!apt-get install p7zip
!p7zip -d -f -k ../input/mercari-price-suggestion-challenge/train.tsv.7z
!unzip -o ../input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip -o ../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip
!p7zip -d -f -k ../input/mercari-price-suggestion-challenge/test.tsv.7z

In [ ]:
!pip install autogluon

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_csv('train.tsv', sep='\t').sample(frac=0.015).reset_index()

In [ ]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [ ]:
from autogluon.text import TextPredictor
import pandas as pd

train_data = pd.read_csv('sub_train.csv')
test_data = pd.read_csv('sub_val.csv')
test_data_nolab = test_data.drop(columns=['price']) 
time_limit = 1 * 60  # set to larger value in your applications
predictor = TextPredictor(label='price', path='autogluon')
predictor.fit(train_data, time_limit=time_limit)
y_pred = predictor.predict(test_data_nolab)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
RMSE = np.sqrt(mean_squared_error(test_data['price'], y_pred))
RMSE